# Evaluating Numerical Expressions

In this project, we will use stacks to implement an algorithm that can evaluate numerical expressions.

However, calculating the result of a complex numerical expression isn't something that a computer processor can do right out of the box. Behind the scenes, Python uses an algorithm to evaluate this expression.

The goal of this project is to use the stack data structure that we've worked with before to implement an algorithm that can evaluate complex numerical expressions.

By the end of this project, we'll know how to implement a function named `evaluate()` that can evaluate expressions stored in string.

We start by loading the previously written `LinkedList` implementation and also adding the implementation for the `Stack`class.

In [1]:
from linked_list import LinkedList

In [2]:
class Stack(LinkedList):

    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.head = self.tail = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

## Infix and Postfix Notation

When we write an expression, we use **infix notation**, meaning that we put the operators between the two operands. For example `1 + 2` is in infix notation because the `+` operator is between the operands `1` and `2`.

For a computer, it's much easier to evaluate an expression written in **postfix notation**. In postfix notation, the operands appear before the operator. The infix expression `1 + 2` becomes `1 2 +` in postfix notation.

We can evaluate an expression in postfix notation using a stack. We read the expression from left to right and do the following:

1. If we find a number, we push that number to the top of the stack.
2. If we find an operator, we pop the top two elements of the stack, perform the operation, and then push back the result.

After processing the entire expression, there will be a single element on the stack. This value is the result of the operation.

Let's implement a function `evaluate_postfix()` that evaluates an expression in postfix notation. To simplify the function, we will assume that, in the expression string we want to evaluate, there are spaces between all elements of the expression. Based on this assumption, we can transform the postfix expression string into a list of elements using the `str.split()` method.

In the context of evaluating expressions, we call these elements **tokens**, and the term for transforming the expression into a list of tokens is **tokenize**.

Let's write a function that tokenizes a postfix expression using the `str.split()` method.

In [3]:
def tokenize(exp):
    return exp.split()

In [4]:
print(tokenize("12 2 4 + / 21 *"))

['12', '2', '4', '+', '/', '21', '*']


## Processing an Operator

We learned that we can evaluate an expression in postfix notation using a stack. The idea is that we read the expression from left to right and do the following:

1. If we find a number, then we push that number to the top of the stack.
2. If we find an operator, we pop the top two elements of the stack, perform the operation, and then push back the result.

There is one important detail we need to consider in the second step. When we find an operator, we pop the top two values on the top of the stack. When we apply the operator to those two elements, we need to make sure we operate those two numbers in the correct order. Let's write this functions for each operator `-`, `+`, `*`, `/`, `**`.

In [5]:
# process minus
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)

In [6]:
# process plus
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)

In [7]:
# process times
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)

In [8]:
# process divide
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)

In [9]:
# process power
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

## Evaluating Postfix Expressions

At this point, we have functions to do the following:

- Transform the expression into a list of tokens
- Process each of the five operators `+`, `-`, `*`, `/`, and `**`

We can now implement an algorithm to evaluate an expression in postfix notation. To do so, we need to do the following:

1. Tokenize the expression using the `tokenize()` function
2. Initialize an empty stack
3. For each token, do the following:
    1. If the token is an operator, call the corresponding function to process it. For example, if we find a `+` , we call the `process_plus()` function.
    2. Otherwise (the token is a number) we push that number to the top of the stack. Since each token is a string, we'll need to convert it to a `float` first.
4. Return the value that is left in the stack.

In [10]:
# evaluate_postfix function
def evaluate_postfix(exp):
    elements = tokenize(exp)
    stack = Stack()
    for elem in elements:
        if elem == '-':
            process_minus(stack)
        elif elem == '+':
            process_plus(stack)
        elif elem == '*':
            process_times(stack)
        elif elem == '/':
            process_divide(stack)
        elif elem == '**':
            process_pow(stack)
        else:
            stack.push(float(elem))
    return stack.pop()

In [11]:
# testing the function
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for exp in expressions:
    print(evaluate_postfix(exp))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


That seems to work alright!

## Operator Precedence in Infix Notation

Great! We can now evaluate postfix expressions! But to make this project useful, we need to enable our algorithm to evaluate expressions in infix notation. After all, it would be very annoying to have to write expressions in postfix notation to use our algorithm. Like before, to simplify tokenizing the expression, we'll assume that the infix expression string contains spaces between any two tokens (even the parentheses). 

To convert an expression from infix to postfix, we'll implement the [Shunting-yard algorithm](https://en.wikipedia.org/wiki/Shunting_yard_algorithm). The data structure to implement this algorithm is (again) a stack.

In the Shunting-yard algorithm, we'll need to compare the precedence of the operators (i.e. which operator has to be executed first). We will use numbers to represent the operator precedence. The higher the precedence, the higher the number. For this, we will create a dictionary.

In [12]:
# precedence dictionary

precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
    "**": 3    
}

In [13]:
# test the precedence dictionary
print(precedence["/"] < precedence["-"])
print(precedence["**"] > precedence["*"])
print(precedence["+"] == precedence["-"])

False
True
True


## From Infix to Postfix

Now we'll implement a function `infix_to_postfix()` that, given a string with an expression in infix notation, outputs a string with that expression written in postfix notation.

This function will implement the Shunting-yard algorithm. This algorithm is similar to the `evaluate_postfix()` function we've implemented before. It starts by tokenizing the postfix expression, and then it processes the tokens one by one using a stack. It builds the postfix expression by keeping track of a list named postfix, which will contain the list of tokens in postfix order. 

The function needs to process opening and closing prentheses, operators, and numbers. We will implement everything step by step, starting with the open parenthesis.

In [14]:
# function to process opening parenthesis
def process_opening_parenthesis(stack):
    stack.push("(")

## Handling Closing Parenthesis

In this step we'll write a function for handling a closing parenthesis. Here's how the algorithm specification said we should handle this case:

- Closing parenthesis `)`:
1. While the top of the stack isn't an opening parenthesis, `(`, pop the top element, and append it to the `postfix` token list.
2. Pop the opening parentheses out of the stack at the end.

In [15]:
# function to process closing parenthesis
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != "(":
        element = stack.pop()
        postfix.append(element)
    stack.pop()

## Handling Operators

Next we will write a function to handle the operators:

- Operator, `+`, `-`, `*`, / or `**`:
    - While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element and append it to the postfix token list.
    - Push the current operator to the top of the stack.

Above, we see again that we need to use a while loop. The condition of the while loop needs to check that the top of the stack is an operator and that its precedence is greater than or equal to the precedence of the operator we're processing.

We can get the top of the stack (without removing it) using the `Stack.peek()` function. However, we first need to ensure that the stack isn't empty, or it will cause an error.

Earlier, we defined a dictionary named `precedence` to compare the precedence of two operators. Here we will use this now.

In [23]:
# function to handle operators
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        element = stack.pop()
        postfix.append(element)
    stack.push(operator)

## Handling Numbers

The only type of token that remains to process is numbers. Here's how the algorithm specification said we should handle numbers:

- Operand (any number):
    1. Push the token into the postfix token list.

Let's implement this.

In [17]:
# function to process numbers
def process_number(postfix, number):
    postfix.append(number)

##  Implementing the Shunting-yard Algorithm   

We now have all the pieces we need to implement the `infix_to_postfix()` function that converts an expression from infix notation to postfix notation.

This function will work as follows:

1. We start by splitting the expression into tokens using the `tokenize()` function.
2. We initialize an empty stack.
3. We initialize an empty postfix token list.
4. Iterate over all tokens, and for each, do the following:
    - If the token is `"("`, we call the `process_opening_parenthesis()` function.
    - If the token is `")"`, we call the `process_closing_parenthesis()` function.
    - If the token is an operator, we call the `process_operator()` function.
    - Otherwise, the token is a number, and we call the `process_number()` function.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the `str.join()` method to convert the postfix token list into a string.

Let's put all the piece together and implement the `infix_to_postfix()` function.

In [25]:
# infix to postfix function
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in ["-", "+", "*", "/", "**"]:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return " ".join(postfix)            

## Evaluating Infix expressions

We now have a function that can transform an infix expression into postfix notation and a function that can evaluate an expression in postfix notation. By combining the two, we can write a function named `evaluate()` that returns the value of an expression in infix notation.

Let's finish this project by implementing the `evaluate()` function.

In [21]:
# evaluate function
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

In [26]:
# testing the evaluate function
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for exp in expressions:
    print(evaluate(exp))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0


That looks good!